# Reference experiment visualization

This notebook shows some visualization of the two reference experiements described in chapter 5.1 of [this paper](https://arxiv.org/abs/2001.00593). 

### Import  stuff

In [ ]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [ ]:
from comm_agents.data.reference_experiments import RefExperimentMass, RefExperimentCharge
from comm_agents.data.optimal_answers import get_alpha_star, get_phi_star
from ipywidgets import interact
import numpy as  np

### Define params

In [ ]:
# experiment parameters
GOLF_HOLE_LOC_M = .1
GOLF_HOLE_LOC_C = .1
TOLERANCE = .01
PARAM_DICT = dict(
    m=[1e-20, 1e-20],
    q=[1e-16, -1e-15],
    m_ref_m=2e-20,
    q_ref=[-1e-17, -1e-17],
    v_ref_m=2,
    m_ref_c=2e-20,
    v_ref_c=0,
    N=10,
    alpha=[0, 0],
    phi=[0, 0],
    dt=.01,
    d=.1,
    is_golf_game=False,
    gravity=True)

# ranges for interative plots
alpha_range_re_a = np.linspace(0, 0.6, 50)
m_range = list(np.linspace(1e-20, 1e-19, 50))
alpha_range_re_b = np.linspace(0.5, 1, 20)
q_range_re_b_1 = np.linspace(1e-16, 2e-16, 50)
q_range_re_b_2 = np.linspace(-1e-15, -2e-15, 50)
v_ref_range = np.linspace(1, 2, 50)
N_reange = [10, 100, 1000]
dt_range = [.01, .001, .0001]

### Interactive visualization for reference experiment A

In [ ]:
def get_re_a_plot(alpha1, alpha2, m1, m2, v_ref, N, dt, alpha_star):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.dt = dt
    rem.angle = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.v_ref = v_ref
    if alpha_star:
        rem.angle, loss = get_alpha_star(rem)
        hio = rem.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    rem.set_initial_state()
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
    
interact(get_re_a_plot, alpha1=alpha_range_re_a, alpha2=alpha_range_re_a,
         m1=m_range, m2=m_range, N=N_reange, alpha_star=False, v_ref=v_ref_range,
        dt=dt_range)

### Interactive visualization for reference experiment B

In [ ]:
def get_re_b_plot(m1, m2, v_ref, phi1=.5, phi2=.63, q1=4e-12, q2=-3e-12, N=100, 
                 dt=0.01, phi_star=False, y_cap=True):
    req = RefExperimentCharge(**PARAM_DICT)
    req.is_golf_game = True
    req.N = N
    req.dt = dt
    req.y_cap = y_cap
    req.angle = np.array([phi1*np.pi, phi2*np.pi])
    req.q = np.array([q1, q2])
    req.m = np.array([m1, m2])
    req.v_ref = v_ref
    if phi_star:
        req.angle, loss = get_phi_star(req)
        hio = req.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_C, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    req.set_initial_state()
    req.run()
    req.visualize(golf_hole_loc=GOLF_HOLE_LOC_C)

interact(get_re_b_plot, phi1=alpha_range_re_a, phi2=alpha_range_re_b,
        q1=q_range_re_b_1, q2=q_range_re_b_2,
         m1=m_range, m2=m_range,
         N=N_reange, phi_star=False,
         v_ref=v_ref_range,
         y_cap=True,
        dt=dt_range)